In [1]:
import numpy as np
import csv
import math
import pandas as pd
import os


##### Sharp Ratio

In [2]:
def find_sharpa_ratio(daily_profit):
    sum_profit = sum(daily_profit)
    average = sum_profit / len(daily_profit)
    variance = sum((x - average) ** 2 for x in daily_profit) / len(daily_profit)
    standard_deviation = math.sqrt(variance)
    sharpa_ratio = average / standard_deviation if standard_deviation != 0 else 0
    return sharpa_ratio

In [3]:
finance = "XLE"

In [4]:
# Define la ruta raíz desde donde comenzar a recorrer las carpetas
root_path = f'C:/Users/Usuario/Desktop/Maestria/FinTech/CNN2/data/{finance}'

In [3]:
root_path = "C:/Users/Usuario/Desktop/Maestria/FinTech/CNN2/Data/QQQ/Backtesting/"
data = pd.read_csv(f'{root_path}/test.csv').values.tolist()

In [4]:
data[1][2]

1.0

In [5]:

daily_profit = np.zeros(len(data) - 1)
money = 10000.0
money_bah = 10000.0
total_gain = 0.0
maximum_money = 0.0
minimum_money = 10000.0
maximum_gain = 0.0
maximum_loss = 100.0
total_percent_profit = 0.0
transaction_count = 0
success_transaction_count = 0
failed_transaction_count = 0
total_transaction_length = 0
builder = []

print("Start Capital: $", money)
builder.append(f"Start Capital: ${money}\n")

k = 0
while k < len(data) - 1:
    daily_profit[k] = 0.0
    if float(data[k][2]) == 1:
        buy_point = float(data[k][1]) * 100
        share_number = (money - 1.0) / buy_point
        force_sell = False
        for j in range(k, len(data) - 1):
            sell_point = float(data[j][1]) * 100
            money_temp = (share_number * sell_point) - 1.0
            
            if float(data[j][2]) == 2 or force_sell:
                sell_point = float(data[j][1]) * 100
                gain = sell_point - buy_point
                if gain > 0:
                    success_transaction_count += 1
                else:
                    failed_transaction_count += 1

                if gain >= maximum_gain:
                    maximum_gain = gain
                    maximum_profit_percent = maximum_gain / buy_point * 100

                if gain <= maximum_loss:
                    maximum_loss = gain
                    maximum_loss_percent = maximum_loss / buy_point * 100

                moneyTemp=(share_number*sell_point)-1.0
                money = money_temp
                if money > maximum_money:
                    maximum_money = money
                if money < minimum_money:
                    minimum_money = money

                transaction_count += 1
                profit = round(gain * share_number, 2)
                capital = round(money, 2)
                print(f"{transaction_count}.({k+1}-{j+1}) => {profit} Capital: ${capital}")
                builder.append(f"{transaction_count}.({k+1}-{j+1}) => {profit} Capital: ${capital}\n")
                
                prof = round(profit / (money - profit), 4)
                number_of_day = j - k
                one_day_prof = round(prof / number_of_day, 4)
                for m in range(k+1, j+1):
                    daily_profit[m] = one_day_prof
                
                total_percent_profit += gain / buy_point
                total_transaction_length += j - k
                k = j + 1
                total_gain += gain
                break

    k += 1
for z in range(len(daily_profit)):
    print(f"{z}: {daily_profit[z]}")

sharpa_ratio = find_sharpa_ratio(daily_profit)
print(f"Sharpa Ratio of Our System => {sharpa_ratio}")
builder.append(f"Sharpa Ratio of Our System => {sharpa_ratio}\n")
print(f"Our System => totalMoney = ${round(money, 2)}")
builder.append(f"Our System => totalMoney = ${round(money, 2)}\n")

buy_point_bah = float(data[0][1])
share_number_bah = (money_bah - 1.0) / buy_point_bah
money_bah = (float(data[-1][1]) * share_number_bah) - 1.0
print(f"BAH => totalMoney = ${round(money_bah, 2)}")
builder.append(f"BAH => totalMoney = ${round(money_bah, 2)}\n")


number_of_days = len(data) - 1
number_of_years = number_of_days / 365

print("Our System Annualized return % => {:.2f}%".format((math.pow(money / 10000.0, 0.2) - 1) * 100))  # 5 años, 0.2
print("BaH Annualized return % => {:.2f}%".format((math.exp(math.log(money_bah / 10000.0)) - 1) * 100))
print("Annualized number of transactions => {:.1f}#".format(float(transaction_count)))
print("Percent success of transactions => {:.2f}%".format((float(success_transaction_count) / float(transaction_count) if transaction_count > 0 else 0) * 100))
print("Average percent profit per transaction => {:.2f}%".format((total_percent_profit / transaction_count if transaction_count > 0 else 0) * 100))
print("Average transaction length => {}#".format(total_transaction_length / transaction_count if transaction_count > 0 else 0))
print("Maximum profit percent in transaction => {:.2f}%".format(maximum_profit_percent))
print("Maximum loss percent in transaction => {:.2f}%".format(maximum_loss_percent))
print("Maximum capital value => ${:.2f}".format(maximum_money))
print("Minimum capital value => ${:.2f}".format(minimum_money))
print("Idle Ratio % => {:.2f}%".format((float(len(data) - total_transaction_length) / float(len(data))) * 100))

builder.append("Our System Annualized return % => {:.2f}%\n".format((math.pow(money / 10000.0, 0.2) - 1) * 100))  # 5 años, 0.2
# builder.append("Our System Annualized return % => {:.2f}%\n".format((math.exp(math.log(money / 10000.0)) - 1) * 100))
builder.append("BaH Annualized return % => {:.2f}%\n".format((math.exp(math.log(money_bah / 10000.0)) - 1) * 100))
builder.append("Annualized number of transactions => {:.1f}#\n".format(float(transaction_count)))
builder.append("Percent success of transactions => {:.2f}%\n".format((float(success_transaction_count) / float(transaction_count) if transaction_count > 0 else 0) * 100))
builder.append("Average percent profit per transaction => {:.2f}%\n".format((total_percent_profit / transaction_count if transaction_count > 0 else 0) * 100))
builder.append("Average transaction length => {}#\n".format(total_transaction_length / transaction_count if transaction_count > 0 else 0))
builder.append("Maximum profit percent in transaction => {:.2f}%\n".format(maximum_profit_percent))
builder.append("Maximum loss percent in transaction => {:.2f}%\n".format(maximum_loss_percent))
builder.append("Maximum capital value => ${:.2f}\n".format(maximum_money))
builder.append("Minimum capital value => ${:.2f}\n".format(minimum_money))
builder.append("Idle Ratio % => {:.2f}%\n".format((float(len(data) - total_transaction_length) / float(len(data))) * 100))

with open(f'{root_path}/ResultsNoStop.txt', "w") as writer:
    writer.write("".join(builder))

Start Capital: $ 10000.0
1.(2-6) => 271.47 Capital: $10269.47
2.(8-9) => 53.1 Capital: $10320.57
3.(12-18) => 106.83 Capital: $10425.4
4.(24-31) => -286.51 Capital: $10136.89
5.(34-37) => 360.42 Capital: $10495.31
6.(39-41) => 62.4 Capital: $10555.71
7.(44-45) => -86.82 Capital: $10466.89
8.(47-54) => 568.57 Capital: $11033.46
9.(57-64) => 163.03 Capital: $11194.49
10.(66-69) => 210.67 Capital: $11403.16
11.(72-79) => 149.65 Capital: $11550.81
12.(81-90) => -52.0 Capital: $11496.81
13.(95-108) => -233.87 Capital: $11260.94
14.(110-111) => -26.96 Capital: $11231.98
15.(116-118) => 72.84 Capital: $11302.82
16.(120-123) => -1.17 Capital: $11299.64
17.(125-126) => 4.98 Capital: $11302.63
18.(129-136) => -68.03 Capital: $11232.6
19.(140-145) => 34.09 Capital: $11264.68
20.(150-154) => 82.91 Capital: $11345.59
21.(156-158) => -19.28 Capital: $11324.31
22.(161-163) => 35.5 Capital: $11357.81
23.(165-177) => -26.87 Capital: $11328.94
24.(179-188) => 323.81 Capital: $11650.76
25.(192-198) => 96

In [7]:
for root, dirs, files in os.walk(f'{root_path}/dataset'):
    dirs[:] = [d for d in dirs if d not in ('training', 'testing')]
    # Imprimir los nombres de todas las carpetas en el directorio actual
    for dir_name in dirs:
        data = pd.read_csv(f'{os.path.join(root, dir_name)}/testing/images/cnn_result.csv').values.tolist()
        daily_profit = np.zeros(len(data) - 1)
        money = 10000.0
        total_gain = 0.0
        maximum_money = 0.0
        minimum_money = 10000.0
        maximum_gain = 0.0
        maximum_loss = 100.0
        total_percent_profit = 0.0
        transaction_count = 0
        success_transaction_count = 0
        failed_transaction_count = 0
        total_transaction_length = 0
        builder = []

        print("Start Capital: $", money)
        builder.append(f"Start Capital: ${money}\n")

        k = 0
        while k < len(data) - 1:
            daily_profit[k] = 0.0
            if float(data[k][2]) == 1:
                buy_point = float(data[k][3]) * 100
                share_number = (money - 1.0) / buy_point
                force_sell = False
                for j in range(k, len(data) - 1):
                    sell_point = float(data[j][3]) * 100
                    money_temp = (share_number * sell_point) - 1.0
                    
                    if float(data[j][2]) == 2 or force_sell:
                        gain = sell_point - buy_point
                        if gain > 0:
                            success_transaction_count += 1
                        else:
                            failed_transaction_count += 1

                        if gain >= maximum_gain:
                            maximum_gain = gain
                            maximum_profit_percent = maximum_gain / buy_point * 100

                        if gain <= maximum_loss:
                            maximum_loss = gain
                            maximum_loss_percent = maximum_loss / buy_point * 100

                        money = money_temp
                        if money > maximum_money:
                            maximum_money = money
                        if money < minimum_money:
                            minimum_money = money

                        transaction_count += 1
                        profit = round(gain * share_number, 2)
                        capital = round(money, 2)
                        print(f"{transaction_count}.({k+1}-{j+1}) => {profit} Capital: ${capital}")
                        builder.append(f"{transaction_count}.({k+1}-{j+1}) => {profit} Capital: ${capital}\n")
                        
                        prof = round(profit / (money - profit), 4)
                        number_of_day = j - k
                        one_day_prof = round(prof / number_of_day, 4)
                        for m in range(k+1, j+1):
                            daily_profit[m] = one_day_prof
                        
                        total_percent_profit += gain / buy_point
                        total_transaction_length += j - k
                        k = j + 1
                        total_gain += gain
                        break

            k += 1
        for z in range(len(daily_profit)):
            print(f"{z}: {daily_profit[z]}")

        sharpa_ratio = find_sharpa_ratio(daily_profit)
        print(f"Sharpa Ratio of Our System => {sharpa_ratio}")
        builder.append(f"Sharpa Ratio of Our System => {sharpa_ratio}\n")
        print(f"Our System => totalMoney = ${round(money, 2)}")
        builder.append(f"Our System => totalMoney = ${round(money, 2)}\n")

        buy_point_bah = float(data[0][3])
        share_number_bah = (10000.0 - 1.0) / buy_point_bah
        money_bah = (float(data[-1][3]) * share_number_bah) - 1.0
        print(f"BAH => totalMoney = ${round(money_bah, 2)}")
        builder.append(f"BAH => totalMoney = ${round(money_bah, 2)}\n")


        number_of_days = len(data) - 1
        number_of_years = number_of_days / 365

        print("Our System Annualized return % => {:.2f}%".format((math.pow(money / 10000.0, 1) - 1) * 100))  # 5 años, 0.2
        print("BaH Annualized return % => {:.2f}%".format((math.exp(math.log(money_bah / 10000.0)) - 1) * 100))
        print("Annualized number of transactions => {:.1f}#".format(float(transaction_count)))
        print("Percent success of transactions => {:.2f}%".format((float(success_transaction_count) / float(transaction_count) if transaction_count > 0 else 0) * 100))
        print("Average percent profit per transaction => {:.2f}%".format((total_percent_profit / transaction_count if transaction_count > 0 else 0) * 100))
        print("Average transaction length => {}#".format(total_transaction_length / transaction_count if transaction_count > 0 else 0))
        print("Maximum profit percent in transaction => {:.2f}%".format(maximum_profit_percent))
        print("Maximum loss percent in transaction => {:.2f}%".format(maximum_loss_percent))
        print("Maximum capital value => ${:.2f}".format(maximum_money))
        print("Minimum capital value => ${:.2f}".format(minimum_money))
        print("Idle Ratio % => {:.2f}%".format((float(len(data) - total_transaction_length) / float(len(data))) * 100))

        builder.append("Our System Annualized return % => {:.2f}%\n".format((math.pow(money / 10000.0, 1) - 1) * 100))  # 5 años, 0.2
        # builder.append("Our System Annualized return % => {:.2f}%\n".format((math.exp(math.log(money / 10000.0)) - 1) * 100))
        builder.append("BaH Annualized return % => {:.2f}%\n".format((math.exp(math.log(money_bah / 10000.0)) - 1) * 100))
        builder.append("Annualized number of transactions => {:.1f}#\n".format(float(transaction_count)))
        builder.append("Percent success of transactions => {:.2f}%\n".format((float(success_transaction_count) / float(transaction_count) if transaction_count > 0 else 0) * 100))
        builder.append("Average percent profit per transaction => {:.2f}%\n".format((total_percent_profit / transaction_count if transaction_count > 0 else 0) * 100))
        builder.append("Average transaction length => {}#\n".format(total_transaction_length / transaction_count if transaction_count > 0 else 0))
        builder.append("Maximum profit percent in transaction => {:.2f}%\n".format(maximum_profit_percent))
        builder.append("Maximum loss percent in transaction => {:.2f}%\n".format(maximum_loss_percent))
        builder.append("Maximum capital value => ${:.2f}\n".format(maximum_money))
        builder.append("Minimum capital value => ${:.2f}\n".format(minimum_money))
        builder.append("Idle Ratio % => {:.2f}%\n".format((float(len(data) - total_transaction_length) / float(len(data))) * 100))

        with open(f'{os.path.join(root, dir_name)}/testing/ResultsNoStop4.txt', "w") as writer:
            writer.write("".join(builder))

Start Capital: $ 10000.0
1.(11-19) => 99.67 Capital: $10097.67
2.(24-28) => 153.66 Capital: $10249.33
3.(30-34) => 203.6 Capital: $10450.94
4.(39-40) => -89.36 Capital: $10359.58
5.(45-48) => 84.83 Capital: $10442.41
6.(59-60) => -4.24 Capital: $10436.17
7.(62-64) => 42.77 Capital: $10476.93
8.(70-73) => -21.82 Capital: $10453.11
9.(76-80) => -236.76 Capital: $10214.35
10.(87-97) => 356.59 Capital: $10568.94
11.(105-109) => -167.54 Capital: $10399.4
12.(118-127) => -87.39 Capital: $10310.01
13.(129-133) => 128.33 Capital: $10436.34
14.(142-143) => -125.48 Capital: $10308.86
15.(150-151) => -0.0 Capital: $10306.86
16.(153-162) => -42.59 Capital: $10262.28
17.(169-175) => 263.98 Capital: $10524.26
18.(177-182) => 459.75 Capital: $10982.01
19.(184-185) => -67.14 Capital: $10912.87
0: 0.0
1: 0.0
2: 0.0
3: 0.0
4: 0.0
5: 0.0
6: 0.0
7: 0.0
8: 0.0
9: 0.0
10: 0.0
11: 0.0013
12: 0.0013
13: 0.0013
14: 0.0013
15: 0.0013
16: 0.0013
17: 0.0013
18: 0.0013
19: 0.0
20: 0.0
21: 0.0
22: 0.0
23: 0.0
24: 0